# SageMaker Pipelines with Amazon SageMaker Geospatial Capabilities


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

---


This notebook demonstrates how to create a SageMaker Pipeline to automate geospatial data processing workflows.

Amazon SageMaker Pipelines is a tool for building machine learning pipelines that take advantage of direct SageMaker integration. In this notebook, we will combine Amazon SageMaker geospatial capabilities with Amazon SageMaker Pipelines to create a reproducible and automated workflow which executes a chain of Earth Observation Jobs.

![Pipeline Architecture](images/pipeline_architecture.png)

----

## Prerequisites

### General setup

This notebook runs with Kernel Geospatial 1.0. Note that the following policies need to be attached to the execution role that you used to run this notebook:

- AmazonSageMakerFullAccess
- AmazonSageMakerGeospatialFullAccess

You can see the policies attached to the role in the IAM console under the permissions tab. If required, add the roles using the 'Add Permissions' button.

### Interactions with AWS Lambda, Amazon S3, and Amazon SQS

To allow deployment of the Lambda function and use the SageMaker geospatial services from within the notebook, the notebook's execution role needs to allow to assume the role. This can be done by adding the following trust policy using the 'Trust relationships' tab:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "sagemaker.amazonaws.com",
                    "sagemaker-geospatial.amazonaws.com",
                    "lambda.amazonaws.com",
                    "s3.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

Additionaly the notebook's AWS IAM role needs to have the proper permissions for interacting with AWS Lambda, Amazon S3, and Amazon SQS as required.

For demo purposes, the following inline policy can be added. Please note that this policy should be scoped down to improve security for any production deployment, following the least privilege principle:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:DeleteObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "sqs:SendMessage",
                "sqs:ReceiveMessage",
                "sqs:DeleteMessage",
                "sqs:GetQueueAttributes",
                "sqs:GetQueueUrl",
                "sqs:CreateQueue"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "lambda:*",
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:GetRole",
                "iam:PassRole",
                "sts:GetCallerIdentity"
            ],
            "Resource": "*"
        }
    ]
}

```

-----
## Import SageMaker geospatial capabilities SDK

In [ ]:
import boto3
import sagemaker
import json
from datetime import datetime

session = boto3.Session()
execution_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = "us-west-2"
bucket = sagemaker_session.default_bucket()  # Alternatively you can use your custom bucket here.
bucket_prefix = "eoj-pipeline-example"

--------
## Setup Infrastructure

The following steps will setup the necessary infrastructure to execute a [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) using geospatial capabilities. Earth Observation Jobs (EOJs) are executed asynchronous and the pipeline needs to be able to check the status via callbacks.

For this, we'll setup a Lambda function and a SQS queue which are both later used in the pipeline.

This infrastructure needs to be created only once and can then be referenced in multiple pipelines.

### Create Lambda layer

Before we create the Lambda function, we'll create a [Lambda layer](https://docs.aws.amazon.com/lambda/latest/dg/configuration-layers.html) which contains a recent boto3 version which is necessary to use the sagemaker-geospatial API.

The following steps are packaging the layer code, uploading the code package to S3 and creating the actual Lambda layer.

In [ ]:
import argparse
import os
import subprocess
import sys
import tempfile
import venv
import zipfile


def zipdir(path, zip_fhandle):
    for root, dirs, files in os.walk(path):
        for file in files:
            zip_fhandle.write(
                os.path.join(root, file),
                os.path.relpath(os.path.join(root, file), os.path.join(path, "..")),
            )


requirements = ["boto3==1.26.100"]

output_file = "assets/boto_layer.zip"

tmp_dir = tempfile.TemporaryDirectory()
tmp_dir_path = tmp_dir.name

lambda_dir = os.path.join(tmp_dir_path, "python")
os.mkdir(lambda_dir)

work_dir = os.getcwd()
os.chdir(tmp_dir_path)

venv_dir = os.path.join(tmp_dir_path, "venv/")
pip_bin = os.path.join(venv_dir, "bin/pip")

venv.create(venv_dir, with_pip=True)

for r in requirements:
    subprocess.run([pip_bin, "install", "--quiet", "--disable-pip-version-check", r], check=True)

os.rename(os.path.join(venv_dir, "lib"), os.path.join(lambda_dir, "lib"))
# rename directory to Lambda runtime target version to enable implicit module import
os.rename(
    os.path.join(lambda_dir, "lib", "python3.10"), os.path.join(lambda_dir, "lib", "python3.9")
)

with zipfile.ZipFile(os.path.join(work_dir, output_file), "w") as zf:
    zipdir(os.path.basename(lambda_dir), zf)

os.chdir(work_dir)
tmp_dir.cleanup()

In [ ]:
layer_archive_object_key = f"{bucket_prefix}/infra/boto_layer.zip"
output_file = "assets/boto_layer.zip"

s3_client = boto3.client("s3")
response = s3_client.upload_file(output_file, bucket, layer_archive_object_key)

In [ ]:
lambda_client = boto3.client("lambda")

response = lambda_client.publish_layer_version(
    LayerName="boto3-1_26_100-layer",
    Content={
        "S3Bucket": bucket,
        "S3Key": layer_archive_object_key,
    },
    CompatibleRuntimes=["python3.9"],
)

layer_arn = response["LayerVersionArn"]
print(f"Created Lambda layer with ARN: {layer_arn}")

### Create Lambda function

After the layer has been created, we will create the actual Lambda function. The code for Lambda function is located at [assets/eoj_pipeline_lambda.py](assets/eoj_pipeline_lambda.py).

The Lambda function will provide functionality to start EOJs and also check their status. It will be used in combination with an Amazon SQS queue.

The following cell will create the Lambda function `geospatial-eoj-pipeline-lambda`.

In [ ]:
import zipfile

with zipfile.ZipFile("./assets/eoj_pipeline_lambda.py.zip", "w") as zf:
    zf.write("./assets/eoj_pipeline_lambda.py", "eoj_pipeline_lambda.py")

with open("./assets/eoj_pipeline_lambda.py.zip", "rb") as f:
    zipped_code = f.read()

lambda_client = boto3.client("lambda")

# set this to True to delete the lambda function named `geospatial-eoj-pipeline-lambda` before deploying the lambda function
# if set to True, this allows the execution of this cell multiple times
delete_before_create = False

if delete_before_create:
    try:
        lambda_client.delete_function(FunctionName="geospatial-eoj-pipeline-lambda")
    # ignore if the function does not exist
    except lambda_client.exceptions.ResourceNotFoundException:
        pass


response = lambda_client.create_function(
    FunctionName="geospatial-eoj-pipeline-lambda",
    Runtime="python3.9",
    Role=execution_role,
    Handler="eoj_pipeline_lambda.lambda_handler",
    Code=dict(ZipFile=zipped_code),
    Layers=[layer_arn],
    Timeout=60,
    Environment={"Variables": {"LOGGING_LEVEL": "INFO"}},
)

function_arn = response["FunctionArn"]
print(f"Created Lambda function with ARN: {function_arn}")

### Create SQS queue

After creating the Lambda function to invoke EOJs, we will create an Amazon SQS queue for asynchronous callbacks.

As Earth Observation Jobs (EOJs) are executed in an asynchronous fashion, the SQS queue is needed to check the EOJ status within the pipeline asynchronously.

The following cell will create an Amazon SQS queue which will be later used in the pipeline.

In [ ]:
# Create SQS queue for handling async processes and callback
sqs_client = boto3.client("sqs", region_name=region)

sqs_client.create_queue(
    QueueName=f"geospatial-eoj-pipeline-queue",
    Attributes={
        "VisibilityTimeout": "300",
        "DelaySeconds": "5",
        "ReceiveMessageWaitTimeSeconds": "5",
    },
)
queue_url = sqs_client.get_queue_url(QueueName=f"geospatial-eoj-pipeline-queue")["QueueUrl"]
queue_arn = sqs_client.get_queue_attributes(QueueUrl=queue_url, AttributeNames=["QueueArn"])[
    "Attributes"
]["QueueArn"]

print("Created queue: {}".format(json.dumps({"url": queue_url, "arn": queue_arn}, indent=4)))

Now we need to associate the SQS queue as an input trigger for our Lambda function, in this way whenever the Callback Step pushes a message to the queue it would run our Lambda function for checking the status of the EOJ. We do this by creating an Event Source Mapping between the Lambda function and the SQS queue.

In [ ]:
# Link SQS queue with the geospatial-check-lambda function...
event_source_mapping = lambda_client.create_event_source_mapping(
    EventSourceArn=queue_arn, FunctionName="geospatial-eoj-pipeline-lambda", Enabled=True
)
print(f'Mapping Lambda function and SQS queue through UUID: {event_source_mapping["UUID"]}')

----
## Pipeline Creation

After the necessary infrastructure has been deployed, we can build the pipeline.

We will define the parameters which are used to configure our pipeline dynamically whenever we execute the pipeline.

The code below allows the creation of a desired amount of EOJ steps within the pipeline. In this example, we will create 3 EOJ steps and one additional export step. By adapting the parameter `eoj_step_count`, the amount of EOJ steps can be changed.

In [ ]:
from sagemaker.workflow.parameters import ParameterString

parameter_execution_role = ParameterString(
    name="parameter_execution_role", default_value=execution_role
)
parameter_region = ParameterString(name="parameter_region", default_value=region)
parameter_queue_url = ParameterString(name="parameter_queue_url", default_value=queue_url)
parameter_eoj_input_config = ParameterString(name="parameter_eoj_input_config", default_value="")

# this argument defines how many EOJ steps should be created for the pipeline
eoj_step_count = 3
parameters_step_eoj_config = []
for step_n in range(1, eoj_step_count + 1):
    parameters_step_eoj_config.append(
        ParameterString(name=f"parameter_step_{step_n}_eoj_config", default_value="")
    )

parameter_eoj_export_config = ParameterString(name="parameter_eoj_export_config", default_value="")

Next, we will create as many EOJ steps, as provided in `eoj_step_count`. Each iteration will create a LambdaStep and a CallbackStep which depends on the former.

In [ ]:
import time
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.callback_step import CallbackStep, CallbackOutput, CallbackOutputTypeEnum
from sagemaker.lambda_helper import Lambda

pipeline_steps = []
previous_lambda_step = None
for step_n in range(1, eoj_step_count + 1):
    if previous_lambda_step is None:
        eoj_input_config = parameter_eoj_input_config
        step_dependencies = []
    else:
        eoj_input_config = previous_lambda_step.properties.Outputs["eoj_arn"]
        step_dependencies = [pipeline_steps[-1].name]

    current_lambda_step = LambdaStep(
        name=f"eoj-processing-step-{step_n}",
        depends_on=step_dependencies,
        lambda_func=Lambda(function_arn=function_arn),
        inputs={
            "role": parameter_execution_role,
            "region": parameter_region,
            "eoj_input_config": eoj_input_config,
            "eoj_config": parameters_step_eoj_config[step_n - 1],
            "eoj_name": f"eoj-pipeline-step-{step_n}",
        },
        outputs=[
            LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String),
            LambdaOutput(output_name="eoj_arn", output_type=LambdaOutputTypeEnum.String),
        ],
    )

    callback_step = CallbackStep(
        name=f"eoj-callback-step-{step_n}",
        depends_on=[current_lambda_step.name],
        sqs_queue_url=parameter_queue_url,
        inputs={
            "role": parameter_execution_role,
            "region": parameter_region,
            "eoj_arn": current_lambda_step.properties.Outputs["eoj_arn"],
        },
        outputs=[
            CallbackOutput(output_name="eoj_status", output_type=CallbackOutputTypeEnum.String),
        ],
    )

    previous_lambda_step = current_lambda_step

    pipeline_steps.append(current_lambda_step)
    pipeline_steps.append(callback_step)

Then, we will create additional steps which enable the export of the EOJ.

In [ ]:
last_lambda_step = pipeline_steps[-2]
last_callback_step = pipeline_steps[-1]

export_lambda_step = LambdaStep(
    name="export-eoj-step",
    depends_on=[last_callback_step.name],
    lambda_func=Lambda(function_arn=function_arn),
    inputs={
        "eoj_arn": last_lambda_step.properties.Outputs["eoj_arn"],
        "role": parameter_execution_role,
        "region": parameter_region,
        "eoj_export_config": parameter_eoj_export_config,
    },
    outputs=[
        LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="eoj_arn", output_type=LambdaOutputTypeEnum.String),
    ],
)

export_callback_step = CallbackStep(
    name="export-eoj-callback",
    depends_on=[export_lambda_step.name],
    sqs_queue_url=parameter_queue_url,
    inputs={
        "role": parameter_execution_role,
        "region": parameter_region,
        "eoj_arn": export_lambda_step.properties.Outputs["eoj_arn"],
    },
    outputs=[
        CallbackOutput(output_name="statusJob", output_type=CallbackOutputTypeEnum.String),
    ],
)

pipeline_steps.append(export_lambda_step)
pipeline_steps.append(export_callback_step)

Finally, we can define our pipeline based on the steps and parameters created before.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = "GeospatialEarthObservationPipeline"

pipeline_parameters = [
    parameter_execution_role,
    parameter_region,
    parameter_queue_url,
    parameter_eoj_input_config,
    parameter_eoj_export_config,
] + parameters_step_eoj_config

pipeline = Pipeline(
    name=pipeline_name,
    parameters=pipeline_parameters,
    steps=pipeline_steps,
)

In [ ]:
# uncomment to see the created pipeline definition as JSON in the notebook
# definition = json.loads(pipeline.definition())
# definition

In [ ]:
pipeline.upsert(role_arn=execution_role)

After the pipeline has been created, you are able to inspect the created pipeline.

For this, you can navigate to the SageMaker Studio Resources tab in the left menu and click on `Pipelines`.

You should be able to see the "GeospatialEarthObservationPipeline" in the list. You can click on it and then navigate to the `Graph` tab to see a visual representation of the created pipeline.

![SageMaker Pipeline](images/sagemaker_eo_pipeline.png)

--------
## Pipeline execution

The following cell is defining the configuration for the pipeline and finally runs a pipeline execution.

If you adapted the `eoj_step_count` in the previous cells, you must ensure that `eoj_configs` contain as many EOJ configurations as you have steps.

In [ ]:
aoi_coordinates = [
    [10.046694766538735, 49.62389264555313],
    [10.046694766538735, 49.4502039714734],
    [10.34697773264952, 49.4502039714734],
    [10.34697773264952, 49.62389264555313],
    [10.046694766538735, 49.62389264555313],
]

eoj_input_config = {
    "RasterDataCollectionQuery": {
        "RasterDataCollectionArn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",  # Sentinel-2 data
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {"PolygonGeometry": {"Coordinates": [aoi_coordinates]}}
        },
        "TimeRangeFilter": {
            "StartTime": "2022-05-01T00:00:00Z",
            "EndTime": "2022-07-31T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 2}}}]
        },
    }
}

eoj_config_step_1 = {
    "CloudRemovalConfig": {
        "AlgorithmName": "INTERPOLATION",
        "InterpolationValue": "-9999",
        "TargetBands": ["red", "nir"],
    }
}

eoj_config_step_2 = {
    "BandMathConfig": {
        "CustomIndices": {
            "Operations": [
                {
                    "Name": "ndvi",
                    "Equation": "(nir - red) / (nir + red)",
                    "OutputType": "FLOAT32",
                },
            ]
        },
    }
}

eoj_config_step_3 = {
    "TemporalStatisticsConfig": {
        "GroupBy": "YEARLY",
        "Statistics": ["MEAN"],
        "TargetBands": ["ndvi"],
    },
}

eoj_configs = [eoj_config_step_1, eoj_config_step_2, eoj_config_step_3]

if len(eoj_configs) != eoj_step_count:
    raise Exception(
        "The number of provided EOJ configs must be the same as the configured EOJ pipeline steps"
    )

eoj_export_config = {"S3Data": {"S3Uri": f"s3://{bucket}/{bucket_prefix}/export/"}}

pipeline_execution_parameters = {
    "parameter_execution_role": execution_role,
    "parameter_region": region,
    "parameter_queue_url": queue_url,
    "parameter_eoj_input_config": eoj_input_config,
    "parameter_eoj_export_config": eoj_export_config,
}

for step_n in range(1, eoj_step_count + 1):
    pipeline_execution_parameters[f"parameter_step_{step_n}_eoj_config"] = eoj_configs[step_n - 1]

execution = pipeline.start(parameters=pipeline_execution_parameters)

After the pipeline execution has been started, you can see it as well in the Pipelines UI.

Navigate back to the "GeospatialEarthObservationPipeline" in the Pipelines UI and you can see the execution in the `Executions` tab. You can double-click it to see the details of this execution.



![SageMaker Pipeline Execution](images/sagemaker_eo_pipeline_execution.png)

### Alternative way to execute the created Pipeline

The Pipeline exection example in the previous cell needs an initialized `Pipeline` object to run a pipeline execution. If you want to execute the pipeline from within other notebooks, this can be done in the following way.

In [ ]:
import boto3
import json

eoj_step_count = 3

# define your pipeline configuration as in the example above
aoi_coordinates = [
    [10.046694766538735, 49.62389264555313],
    [10.046694766538735, 49.4502039714734],
    [10.34697773264952, 49.4502039714734],
    [10.34697773264952, 49.62389264555313],
    [10.046694766538735, 49.62389264555313],
]

eoj_input_config = {
    "RasterDataCollectionQuery": {
        "RasterDataCollectionArn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",  # Sentinel-2 data
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {"PolygonGeometry": {"Coordinates": [aoi_coordinates]}}
        },
        "TimeRangeFilter": {
            "StartTime": "2022-05-01T00:00:00Z",
            "EndTime": "2022-07-31T23:59:59Z",
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 2}}}]
        },
    }
}

eoj_config_step_1 = {
    "CloudRemovalConfig": {
        "AlgorithmName": "INTERPOLATION",
        "InterpolationValue": "-9999",
        "TargetBands": ["red", "nir"],
    }
}

eoj_config_step_2 = {
    "BandMathConfig": {
        "CustomIndices": {
            "Operations": [
                {
                    "Name": "ndvi",
                    "Equation": "(nir - red) / (nir + red)",
                    "OutputType": "FLOAT32",
                },
            ]
        },
    }
}

eoj_config_step_3 = {
    "TemporalStatisticsConfig": {
        "GroupBy": "YEARLY",
        "Statistics": ["MEAN"],
        "TargetBands": ["ndvi"],
    },
}

eoj_configs = [eoj_config_step_1, eoj_config_step_2, eoj_config_step_3]

if len(eoj_configs) != eoj_step_count:
    raise Exception(
        "The number of provided EOJ configs must be the same as the configured EOJ pipeline steps"
    )

eoj_export_config = {"S3Data": {"S3Uri": f"s3://{bucket}/{bucket_prefix}/export/"}}

pipeline_execution_parameters = {
    "parameter_execution_role": execution_role,
    "parameter_region": region,
    "parameter_queue_url": queue_url,
    "parameter_eoj_input_config": eoj_input_config,
    "parameter_eoj_export_config": eoj_export_config,
}

for step_n in range(1, eoj_step_count + 1):
    pipeline_execution_parameters[f"parameter_step_{step_n}_eoj_config"] = eoj_configs[step_n - 1]

# transform the parameter dictionary into a list format
pipeline_execution_parameters_list = []
for key, value in pipeline_execution_parameters.items():
    if type(value) is dict:
        value = json.dumps(value)
    pipeline_execution_parameters_list.append({"Name": key, "Value": value})

# use the sagemaker client to start the pipeline execution with the PipelineName referencing the previously created pipeline
session = boto3.Session()
sagemaker_client = session.client(service_name="sagemaker")
sagemaker_client.start_pipeline_execution(
    PipelineName="GeospatialEarthObservationPipeline",
    PipelineParameters=pipeline_execution_parameters_list,
)

-------

### Clean-up

When the created pipeline and underlying infrastructure is no longer needed, uncomment and run the following cells for deleting the previously created resources.

In [ ]:
# Delete the SQS queue
# sqs_client = boto3.client("sqs", region_name=region)
# sqs_client.delete_queue(
#    QueueUrl=sqs_client.get_queue_url(QueueName=f"geospatial-eoj-pipeline-queue")["QueueUrl"]
# )

In [ ]:
# Delete Lambda function
# lambda_client = boto3.client("lambda", region_name=region)
# lambda_client.delete_function(FunctionName="geospatial-eoj-pipeline-lambda")

In [ ]:
# Delete the SageMaker Pipeline
# sagemaker_client = boto3.client("sagemaker", region_name=region)
# sagemaker_client.delete_pipeline(PipelineName="GeospatialEarthObservationPipeline")

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-geospatial|geospatial-pipeline|geospatial-pipelines.ipynb)
